### 라이브러리 import

In [3]:

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import mlflow
import mlflow.sklearn

In [7]:
iris = load_iris() # 꽃 받침과 꽃 잎 사이즈를 가지고 꽃의 종류를 결정

X = iris.data
Y = iris.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)